In [ ]:
#Jaderná elektrárna Dukovany(두코바니 원자력 발전소)

#https://search.seznam.cz/clanky/?q=Jadern%C3%A1%20elektr%C3%A1rna%20Dukovany
#리눅스 환경에서 코드 실행 중

: 

In [ ]:
import time
import json
import datetime
import random
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager

# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2022.01.01"
END_DATE = "2022.03.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()


🔍 검색 중: 두코바니 원전 (2022.01.01 ~ 2022.03.31)
✅ 198개의 뉴스 기사 링크 및 날짜를 수집 완료! (2022.01.01 ~ 2022.03.31)


📰 뉴스 본문 크롤링 진행:   0%|          | 0/198 [00:00<?, ?it/s]

⚠ 본문을 가져오지 못해 Selenium으로 재시도: http://www.edaily.co.kr/news/newspath.asp?newsid=02273046632268896


📰 뉴스 본문 크롤링 진행:   1%|          | 1/198 [00:07<23:04,  7.03s/it]

⚠ 본문을 가져오지 못해 Selenium으로 재시도: http://www.edaily.co.kr/news/newspath.asp?newsid=03863846632268896


...

⚠ 본문을 가져오지 못해 Selenium으로 재시도: http://www.kookje.co.kr/news2011/asp/newsbody.asp?code=0200&key=20230829.99099009264


📰 뉴스 본문 크롤링 진행:  67%|██████▋   | 109/162 [05:37<02:21,  2.66s/it]

⚠ 본문을 가져오지 못해 Selenium으로 재시도: https://www.asiatoday.co.kr/view.php?key=20230922010014065


📰 뉴스 본문 크롤링 진행:  67%|██████▋   | 109/162 [05:46<02:48,  3.18s/it]


KeyboardInterrupt: 

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2022.04.01"
END_DATE = "2022.06.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2022.07.01"
END_DATE = "2022.09.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2022.10.01"
END_DATE = "2022.12.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2023.01.01"
END_DATE = "2023.03.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2023.04.01"
END_DATE = "2023.06.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2023.07.01"
END_DATE = "2023.09.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2023.10.01"
END_DATE = "2023.12.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2024.01.01"
END_DATE = "2024.03.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2024.04.01"
END_DATE = "2024.06.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2024.07.01"
END_DATE = "2024.09.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
# ✅ 크롤링할 검색어 (변경 가능)
search_query = "두코바니 원전"

# ✅ 크롤링할 기간 설정 (각 코드 실행 시 해당 기간만 변경)
START_DATE = "2024.10.01"
END_DATE = "2024.12.31"

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query, start_date, end_date):
    """네이버 뉴스 검색 페이지에서 기사 링크 크롤링"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))

    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")

    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()

    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        
        last_height = new_height

        # ⏳ 3분(180초) 이상 실행되면 종료
        if time.time() - start_time > 180:
            print("⏳ 무한 스크롤 방지를 위해 크롤링 중단")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []
    retry_count = 3

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        for attempt in range(retry_count):
            try:
                response = requests.get(news["link"], headers=HEADERS, timeout=10)
                if response.status_code != 200:
                    print(f"⚠ HTTP 오류: {response.status_code}")
                    time.sleep(random.uniform(3, 6))
                    continue

                soup = BeautifulSoup(response.text, "html.parser")

                content_selectors = [
                    "div#newsct_article", "div#articleBodyContents", "div.article_view",
                    "div.news_end", "div#articeBody", "div.content_area",
                    "div#newsEndContents", "article"
                ]
                content = ""
                for selector in content_selectors:
                    content_element = soup.select(selector)
                    if content_element:
                        content = " ".join([p.text.strip() for p in content_element])
                        break

                news["content"] = content.strip() if content else "N/A"
                news_data.append(news)

                break  # 성공하면 재시도 종료

            except Exception as e:
                print(f"⚠ 오류 발생 (뉴스 크롤링 실패, {attempt+1}/{retry_count}): {news['link']}, 오류: {e}")
                time.sleep(random.uniform(3, 6))

    return news_data

# ✅ 크롤링 실행
news_list = fetch_naver_news(search_query, START_DATE, END_DATE)
news_data = fetch_news_content(news_list)

# ✅ JSON 파일 저장
output_file = f"naver_news_{START_DATE.replace('.', '')}_{END_DATE.replace('.', '')}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"✅ {START_DATE} ~ {END_DATE} 크롤링 완료! (저장 파일: {output_file})")
driver.quit()

In [ ]:
import json
import os

# ✅ JSON 파일 리스트 가져오기
json_files = [f for f in os.listdir() if f.startswith("naver_news_") and f.endswith(".json")]

# ✅ 모든 JSON 파일 합치기
all_news = []
for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        news_data = json.load(f)
        all_news.extend(news_data)

# ✅ 통합 JSON 파일 저장
output_file = "naver_news_all.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_news, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(all_news)}개의 뉴스 기사를 통합 저장했습니다! (파일명: {output_file})")
